<a href="https://colab.research.google.com/github/tobyloby12/NLP_Bundesreden/blob/main/NLP_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Connecting to notebook

> Mounting notebook to google drive 




> Changing directory so that I can access data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os

path = 'gdrive/MyDrive'
path = os.path.join(path, 'PD NLP project')
os.chdir(path)

#Importing data

> Create a function to take in a link and save it to drive so that I can access it

> Create a function to read through the pdf and create a text file I can use

In [ ]:
import pandas as pd

df_data = pd.read_excel('PD-Projektdatenbank.xlsx')
df_pd = df_data[['Titel', 'Projektbeschreibung', 'Leistungszeitraum']]
df_speeches = pd.read_csv('speeches.csv')

df_speeches.date[2][:-6] == '1949'

date_filter = []
for line in range(len(df_speeches)):
  date = int(df_speeches.date[line][:-6])
  if date > 2000:
    date_filter.append(True)
  else:
    date_filter.append(False)

pd.DataFrame(date_filter)
speeches_date_filtered = df_speeches[date_filter]

#Text cleaning function

In [ ]:
!pip install nltk;
!pip install spacy;
!python -m spacy download de_core_news_sm;

     |████████████████████████████████| 14.9 MB 40.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [ ]:
from nltk.corpus import stopwords
import nltk
from nltk.corpus import stopwords
import nltk
from string import punctuation
from string import digits
import spacy

nltk.download('stopwords')

german_stopwords = stopwords.words('german')
speech_content = df_speeches[['speechContent']]


#setting up stopwords
nltk.download('stopwords')
german_stopwords = stopwords.words('german')
model_de = spacy.load('de_core_news_sm')

# removing umlauts function
def remove_umlauts(word):
  tempWord = word
  tempWord = tempWord.replace('ä', 'ae')
  tempWord = tempWord.replace('ö', 'oe')
  tempWord = tempWord.replace('ü', 'ue')
  tempWord = tempWord.replace('Ä', 'Ae')
  tempWord = tempWord.replace('Ö', 'Oe')
  tempWord = tempWord.replace('Ü', 'Ue')
  tempWord = tempWord.replace('ß', 'ss')
  return tempWord

#remove currency function
def remove_currency(word):
  tempWord = word
  tempWord = tempWord.replace('$', '')
  tempWord = tempWord.replace('€', '')
  tempWord = tempWord.replace('¥', '')
  tempWord = tempWord.replace('₹', '')
  tempWord = tempWord.replace('£', '')
  return tempWord

#lemmatization function
def lemmatizer(text): 
    sent = []
    doc = model_de(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)



#full function to clean text from input text
def data_cleaning(input_text):
  
  #removing punctuation
  remove_pun = str.maketrans('', '', punctuation)
  text_wo_pun = input_text.translate(remove_pun)

  #removing digits
  remove_digits = str.maketrans('', '', digits)
  text_wo_num = text_wo_pun.translate(remove_digits)

  #removing currency
  text_wo_currency = remove_currency(text_wo_num)

  #lemmatization
  text_lemmatized = lemmatizer(text_wo_currency.lower())

  #removing stop words
  text_wo_stop_words = [word for word in text_lemmatized.split() if text_lemmatized.lower() not in german_stopwords_wo_umlaut]

  return text_wo_stop_words

#setting up stopwords
nltk.download('stopwords')
german_stopwords = stopwords.words('german')
german_stopwords_wo_umlaut = []
for word in german_stopwords:
  german_stopwords_wo_umlaut.append(remove_umlauts(word))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Extracting text from pdf

> Creating function to read and write pdf file to drive

> Creating function to read pdf and convert it to text

In [ ]:
!pip install pdfplumber;

     |████████████████████████████████| 45 kB 3.4 MB/s 
     |████████████████████████████████| 5.6 MB 24.2 MB/s 
     |████████████████████████████████| 139 kB 47.2 MB/s 
     |████████████████████████████████| 1.9 MB 35.8 MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.28-py3-none-any.whl size=32240 sha256=c9b49c90829d3f10f3dc9c7912db4533e6ad8d0c435582b8c46f0bfa91b16835
  Stored in directory: /root/.cache/pip/wheels/f2/b1/a0/c0a77b756d580f53b3806ae0e0b3ec945a8d05fca1d6e10cc1
Successfully built pdfplumber


In [ ]:
import os
import urllib.request
import pdfplumber

def download_pdf(pdf_url, filename):
  response = urllib.request.urlopen(pdf_url)
  file = open(filename + ".pdf", 'wb')
  file.write(response.read())
  file.close()

def convert_pdf_to_text(filename):
  with pdfplumber.open(filename + '.pdf') as pdf:
    without_cleaning = ''
    for i in range(len(pdf.pages)-1):
      page = pdf.pages[i]
      if page.extract_text() != None:
        without_cleaning = without_cleaning + page.extract_text()
    return without_cleaning




In [ ]:
pdf_path = 'pdf_files'
path = os.path.join(os.getcwd(), pdf_path)

pdfurl = 'https://dip21.bundestag.de/dip21/btp/15/15014.pdf'
filename = 'pdf_files/testfile'

download_pdf(pdfurl, filename)
without_cleaning = convert_pdf_to_text(filename)

#with_cleaning = data_cleaning(without_cleaning)

#Trying out BERT with hugging face library
https://www.machinecurve.com/index.php/2020/12/21/easy-text-summarization-with-huggingface-transformers-and-machine-learning/

In [ ]:
!pip install transformers;
from transformers import pipeline

with_cleaning_small = ''
for i in range(400):
  with_cleaning_small += with_cleaning[i] + ' '

# Open and read the article
#f = open("article.txt", "r", encoding="utf8")
to_tokenize = with_cleaning

# Initialize the HuggingFace summarization pipeline
summarizer = pipeline("summarization")
summarized = summarizer(to_tokenize, min_length=75, max_length=300)

# Print summarized text
print(summarized)

NameError: ignored

In [ ]:
without_cleaning_split = []
tokens = without_cleaning.split()
token_length = 350
for i in range(len(tokens)//token_length):
  sentance = ''
  for j in range(token_length):
    sentance += tokens[token_length*i + j] + ' '
  without_cleaning_split.append(sentance)

In [ ]:
summary = []
for sentance in without_cleaning_split:
  to_tokenize = sentance
  print('done')
  # Initialize the HuggingFace summarization pipeline
  summarizer = pipeline("summarization")
  summarized = summarizer(to_tokenize, min_length=10, max_length=300)
  summary.append(summarized)
  # Print summarized text


In [ ]:
print(summary)
print(len(summary))

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

to_tokenize = without_cleaning

# Initialize the HuggingFace summarization pipeline
summarizer = pipeline("summarization")
summarized = summarizer(to_tokenize, min_length=75, max_length=300)

# Print summarized text
print(summarized)

#My own version using hugging transformers

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 26.4 MB/s 
     |████████████████████████████████| 636 kB 28.8 MB/s 
     |████████████████████████████████| 895 kB 42.0 MB/s 
     |████████████████████████████████| 3.3 MB 25.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# function to split sentances into paragraphs
import re
from string import ascii_lowercase as alphabet

def split_to_sentance(max_sentance_length, text):
  split_text = re.split(f'[{punctuation}\n]', text)
  split_text = [sentance.strip() for sentance in split_text]
  [split_text.remove(sentance) for sentance in split_text if sentance in ['','f{punctuation}']]
  
  
  
  paragraphs = ['' for i in range(len(text.split())//max_sentance_length + 1)]
  i = 0
  for sentance in split_text:
    sentance_length = len(sentance.split())

    if len(paragraphs[i].split()) < (max_sentance_length - sentance_length):
      paragraphs[i] += ' ' + sentance
    else:
      i += 1
  return paragraphs







In [ ]:
text = split_to_sentance(350, without_cleaning)
len(text[0].split())

349

In [ ]:
from transformers import pipeline

#file = open('summary.txt', 'w')
file.write('')
i = 0
total = len(without_cleaning.split())
for paragraph in text:
  to_tokenize = paragraph
  print(i/total * 100)

  # Initialize the HuggingFace summarization pipeline
  summarizer = pipeline("summarization")
  summarized = summarizer(to_tokenize, min_length=1, max_length=20)
  file = open('summary.txt', 'a')
  file.write(summarized[0]['summary_text'] + '\n')
  i = i + len(paragraph.split())


In [ ]:
file = open('summary.txt', 'r')

text = ''
for line in file:
  text += line + ' '

In [ ]:
len(text.split())

2004

In [ ]:
print(text)

In [ ]:
def summarisation(filename, input_text, total_length, MIN_length = 1, MAX_length = 300):
  file = open(filename, 'w')
  file.write('')
  i = 0
  total = total_length
  for paragraph in input_text:
    to_tokenize = paragraph
    print(i/total * 100)

    # Initialize the HuggingFace summarization pipeline
    summarizer = pipeline("summarization")
    summarized = summarizer(to_tokenize, min_length=MIN_length, max_length=MAX_length)
    file = open(filename, 'a')
    file.write(summarized[0]['summary_text'] + '\n')
    i = i + len(paragraph.split())

In [ ]:
new_summarisation = split_to_sentance(350, text)
summarisation('second_summary.txt', new_summarisation, len(text.split()), 30, 100)

0.0
17.365269461077844
34.78043912175649
52.09580838323353
69.311377245509
86.62674650698602


In [ ]:
file = open('second_summary.txt', 'r')

text_2 = ''
for line in file:
  text_2 += line + ' '

In [ ]:
print(text_2)
print(len(text_2.split()))

 Inhalt Wahl des Abgeordneten Dieter Wiefelsp Antrag der Fraktionen der SPD der CDU CSU des B� The 15 Wahlperiode – 14 Sitzung Berlin Donnerstag . Operation Fox soll schließlich die in keiten abgeschloss Mazedonien darf nicht Beifall bei der SPD und dem Die CDUCSU wird der Verlä
  Steuerreform wollen 2004 und 2005 Entlastungen für Unter Unser Land braucht dringend  des Bundes eingeschlagen Wir brauchen hierzulande in Europa and dieBayerischeSt Die longue übrigens immer noch der Wachstums In unserer Außenwirtschaftspolitik wollen wir Deutsche Gewerkschaftsbund in der begin
  Die CDU CSU macht es entschieden zu Ausfr Die Personal Service Agenturenkönnen auf der Ba Beifall des Im Vermittlungsausschuss wollen Sie mit uns zusam Hartz ist eine revolutionäre Idee – die Mehrheit in BeifAll bei der SPD and dem BÜNDNIS 90 Nord Regierung war wie sie sich prä
  Dr Jürgen Gehb CDU CSU fassungswidrigke Untersuchungsausschuss in allen Phasen des Par n Schindluder with der Verfassung getrieben werde P

#Cosine similarity matching

In [ ]:
!pip install spacy
!python -m spacy download de_core_news_sm

from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words("german")


In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-multilingual-en-de-v2-tmp-trained-scratch')
doc_embedding = model.encode([text_2])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
potential_projects_series = df_pd[['Titel']]
potential_projects = []
potential_projects_embeddings = []
for i in range(len(potential_projects_series)):
  potential_projects.append(potential_projects_series.iloc[i])

for i in range(len(potential_projects)):
  embeddings = model.encode(potential_projects[i])
  potential_projects_embeddings.append(embeddings)

similarities = []
for i in range(len(potential_projects_embeddings)):
  similarities.append(cosine_similarity(doc_embedding, potential_projects_embeddings[i]).astype(float))


In [ ]:
potential_projects_series['Similarities'] = similarities

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
potential_projects_series = potential_projects_series.sort_values('Similarities')

In [ ]:
potential_projects_series.to_csv('projects_similarities.csv')

#Trying with coalition agreements

In [ ]:
from transformers import pipeline
file_url = 'https://www.bundesregierung.de/resource/blob/974430/847984/5b8bc23590d4cb2892b31c987ad672b7/2018-03-14-koalitionsvertrag-data.pdf'
filename = 'pdf_files/19th-coalition'
#download_pdf(file_url, filename)
coalition_text = convert_pdf_to_text(filename)
coalition_text_split = split_to_sentance(300, coalition_text)
summarisation('coalition_summary_second_try.txt', coalition_text_split, len(coalition_text.split()), 1, 15)

NameError: ignored

In [ ]:
file = open('coalition_summary_second_try.txt', 'r')
text = ''
for line in file:
  text += line


len(text.split())

In [ ]:
#need to do cosine similarity on this

#Web scraping to try create dataset with unique PD departments

In [ ]:
departments = ['MB Bundes- und Landesbau', 'MB Gesundheitswesen', 'MB Kommunalberatung', 'CC beschaffung und Vergabe', 'CC Lebensyzklusorientiertes Projektmanagement, Nachhaltigkeit', 'CC Transformation', 'CC Wirtschaftlichkeit und Finanyen',
               'PG Building Information Modeling', 'PG Kommunales E-Government', 'PG Mediyintechnik', 'PG Smart City, Smart Data', 'PG Wohnen und Quartier']